In [1]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

def corrfunc(x,y, ax=None, **kws):
    """Plot the correlation coefficient in the top left hand corner of a plot."""
    r, _ = pearsonr(x, y)
    ax = ax or plt.gca()
    # Unicode for lowercase rho (ρ)
    rho = '\u03C1'
    ax.annotate(f'{rho} = {r:.2f}', xy=(.1, .9), xycoords=ax.transAxes)
	
data = pd.read_csv("Data\CarPrice_Assignment.csv")

data[['symboling']]=data[['symboling']].astype('category')

data['brandName']=data['CarName'].apply(lambda x:x.split(' ')[0].lower().strip())

data.loc[(data['brandName']=='vw' )| (data['brandName']=='vokswagen'),'brandName']='volkswagen'


data.loc[(data['brandName']=='porcshce')|(data['brandName']=='porsche'),'brandName']='porsche'

data.loc[(data['brandName']=='toyouta' ),'brandName']='toyota'

data_num=data.select_dtypes(include=[np.number])

data_num=data_num.iloc[:,1:]

data_cat=data.select_dtypes(exclude=[np.number])

In [ ]:
g = sns.pairplot(data_num)
g.map_lower(corrfunc)


g.savefig('scatter2.png', dpi=300)

In [ ]:
data_num.columns

data_num2=data_num[['wheelbase','carheight','boreratio','stroke','compressionratio','horsepower','peakrpm','citympg','price']]

g2 = sns.pairplot(data_num2)
g2.map_lower(corrfunc)
g2.savefig('scatter3.png', dpi=300)

In [ ]:
fig,ax = plt.subplots(ncols=2,figsize=(16, 6))
sns.boxplot(data = data, x='symboling', y='price',ax=ax[0])
sns.violinplot(data = data, x='symboling', y='price',ax=ax[1])
plt.show()

In [ ]:
fig,ax = plt.subplots(nrows=2,figsize=(20, 10))
sns.boxplot(data = data, x='brandName', y='price',ax=ax[0])
sns.violinplot(data = data, x='brandName', y='price',ax=ax[1])
plt.show()

In [ ]:
fig,ax = plt.subplots(ncols=2,figsize=(16, 6))
sns.boxplot(data = data, x='fueltype', y='price',ax=ax[0])
sns.violinplot(data = data, x='fueltype', y='price',ax=ax[1])
plt.show()

In [ ]:
fig,ax = plt.subplots(ncols=2,figsize=(16, 6))
sns.boxplot(data = data, x='aspiration', y='price',ax=ax[0])
sns.violinplot(data = data, x='aspiration', y='price',ax=ax[1])
plt.show()

In [ ]:
fig,ax = plt.subplots(ncols=2,figsize=(16, 6))
sns.boxplot(data = data, x='doornumber', y='price',ax=ax[0])
sns.violinplot(data = data, x='doornumber', y='price',ax=ax[1])
plt.show()


In [ ]:
fig,ax = plt.subplots(ncols=2,figsize=(16, 6))
sns.boxplot(data = data, x='carbody', y='price',ax=ax[0])
sns.violinplot(data = data, x='carbody', y='price',ax=ax[1])
plt.show()

In [ ]:
fig,ax = plt.subplots(ncols=2,figsize=(16, 6))
sns.boxplot(data = data, x='drivewheel', y='price',ax=ax[0])
sns.violinplot(data = data, x='drivewheel', y='price',ax=ax[1])
plt.show()

In [ ]:
#only 3 cars in rear that of porche. variable might not be that usable

fig,ax = plt.subplots(ncols=2,figsize=(16, 6))
sns.boxplot(data = data, x='enginelocation', y='price',ax=ax[0])
sns.violinplot(data = data, x='enginelocation', y='price',ax=ax[1])
plt.show()

In [ ]:
datax=data[['drivewheel','enginelocation']]

datax['num1']=1

datax.pivot_table(values='num1',index='drivewheel', columns='enginelocation', aggfunc='sum')

In [ ]:

datax=data[['brandName','enginelocation']]
datax['num1']=1

datax.pivot_table(values='num1',index='brandName', columns='enginelocation', aggfunc='sum').dropna()